# Initiation au Parsing

In [ ]:
# si vous utilisez jupyter notebook dans le navigateur, installez gensim 
! pip install gensim

In [ ]:
import gensim.downloader

# Show all available models in gensim-data

print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
model = gensim.downloader.load('glove-twitter-25')

In [ ]:
len(model.index_to_key)

In [ ]:
# Quel va être le mot le plus proche de "sun" ?
model.most_similar('sun', topn=10)

In [ ]:
# Et quel va être le mot le plus proche de "apple" ?
model.most_similar('apple', topn=10)

In [ ]:
# Mais est-ce que "apple" est quand même proche des autres fruits ?(
print(model.similarity('apple', 'orange'))
print(model.similarity('grape', 'orange'))

In [ ]:
# La matrice W(NxV) que nous avons vu dans le cours est la suivante : 
W = model.get_normed_vectors()